In [1]:
from PIL import Image
import numpy as np
import pickle
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from tqdm import tqdm

c:\users\sasha\desktop\thesis\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    return preprocess_input(x, mode='tf')

In [3]:
with open("train.p", "rb") as train_pickle:
    train_images, train_labels = pickle.load(train_pickle)

In [8]:
with open("valid.p", "rb") as valid_pickle:
    valid_images, valid_labels = pickle.load(valid_pickle)

In [4]:
base_model = VGG19(weights='imagenet')

base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [5]:
model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)

In [6]:
extracted_features_train = []
for img in tqdm(train_images):
    preprocessed_image = preprocess_image(img)
    extracted_features_train.append(model.predict(preprocessed_image)[0])

100%|████████████████████████████████████████████████████████████████████████████| 6514/6514 [2:07:13<00:00,  1.17s/it]


In [9]:
with open("train_vgg_19.p", "wb") as train_pickle:
    pickle.dump((np.array(extracted_features_train), train_labels), train_pickle)

In [10]:
extracted_features_valid = []
for img in tqdm(valid_images):
    preprocessed_image = preprocess_image(img)
    extracted_features_valid.append(model.predict(preprocessed_image)[0])

100%|████████████████████████████████████████████████████████████████████████████████| 814/814 [12:15<00:00,  1.11it/s]


In [11]:
with open("valid_vgg_19.p", "wb") as valid_pickle:
    pickle.dump((np.array(extracted_features_valid), valid_labels), valid_pickle)